In [ ]:
!pip install ../input/timm0412/timm-0.4.12-py3-none-any.whl

In [ ]:
import os
import json
import time
from itertools import product

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import cv2

from tqdm import tqdm

import timm
from torch.nn import functional as F

In [ ]:
BASE_DIR = '../input/petfinder-pawpularity-score'
IMAGE_DIR = 'test'
CSV = 'sample_submission.csv'

In [ ]:
sample_df = pd.read_csv(os.path.join(BASE_DIR, CSV))
sample_df.head()

In [ ]:
sample_df.shape, len(os.listdir(os.path.join(BASE_DIR, IMAGE_DIR)))

In [ ]:
def get_valid_augs(img_size=512):
    return A.Compose(
        [
            A.Resize(height=img_size, width=img_size, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0
    )

In [ ]:
class PetFinderDataset(Dataset):
    def __init__(self, df, augs=None):
        self.df = df
        self.augs = augs
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image = self._load_image(self.df["Id"].iloc[index])
        
        # Apply image augmentations if available
        if self.augs:
            image = self.augs(image=image)["image"]

        return image
    
    def _load_image(self, image_id):
        image = cv2.imread(f'{BASE_DIR}/{IMAGE_DIR}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        return image

In [ ]:
test_dataset = PetFinderDataset(
    df = sample_df,
    augs = get_valid_augs(384)
)

test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    pin_memory=False,
    num_workers=4
)

In [ ]:
class ViT(nn.Module):
    def __init__(self, pretrained=False):

        super(ViT, self).__init__()

        self.model = timm.create_model("vit_base_patch32_384", pretrained=pretrained)
        self.model.head = nn.Linear(self.model.head.in_features, 1)

    def forward(self, x):
        return self.model(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = ViT(pretrained=False).to(device)
model.load_state_dict(torch.load("../input/petfinder-my-vision-transformer-pytorch-tpu/model_5e_20210929-0324.pth"))

In [ ]:
def inference(model, iterator):    
    results = []
    model.eval()

    with torch.no_grad():
        for batch in tqdm(iterator):
            batch = batch.to(device)
            pred = model(batch).cpu().detach().numpy()
            results.append(pred)

    return np.array(results)

In [ ]:
preds = np.concatenate(inference(model, test_loader), axis=0)
sample_df['Pawpularity'] = preds
sample_df.head()

In [ ]:
sample_df.to_csv('submission.csv', index=False)